In [1]:
import pandas
import random
import os
import json
import numpy as np
from datetime import datetime, timedelta

# How much data do we want to make?
ROWCOUNT = 1000

## Part 1

### Customer

In [2]:
custFileName = "nameList1000.txt"

ROOT = os.path.dirname(os.getcwd()) + '/resource/' + custFileName

first_names = []
last_names = []

with open(ROOT, 'r') as file:
    rawnames = file.read().split('\n')

for name in rawnames:
    first_names.append(name[:name.find(' ')])
    last_names.append(name[1+name.find(' '):])

In [3]:
'''
--
-- given list of 1st and 2nd names
-- randomize them and duplicate them randomly for repeat customers
--
'''

# Create a list to hold all names
names = []

# Generate unique names (50% of 4000 = 2000 unique names)
for index in range(2000):
    first = random.choice(first_names)
    last = random.choice(last_names)
    name = f"{first} {last}"
    names.append(name)

# Generate repeated names (remaining 50% will be repeats 5-10 times)
for index in range(ROWCOUNT):  # Need 1000 more distinct names, each repeating 5-10 times
    first = random.choice(first_names)
    last = random.choice(last_names)
    name = f"{first} {last}"
    repeat_count = random.randint(5, 10)  # Repeat each name 5 to 10 times
    names.extend([name] * repeat_count)

# Shuffle the final list to mix unique and repeated names
random.shuffle(names)

# Ensure the list is exactly 1000 names
names = names[:ROWCOUNT]

# Convert the list to a single line with commas separating each name
# result = ", ".join(names)

In [4]:
'''
--
-- DF to pandas
--
'''
customerDict = {}
first_names = []
last_names = []

for name in names:
    first_names.append(name[:name.find(' ')])
    last_names.append(name[1+name.find(' '):])

customerDict["First"] = first_names
customerDict["Last"] = last_names

customerDF = pandas.DataFrame.from_dict(data=customerDict)
customerDF


,First,Last
0,Luna,Parker
1,Gianna,Evans
2,Sarah,Parker
3,Natalia,Thompson
4,Bella,Mitchell
...,...,...
995,Riley,Smith
996,Chloe,Cruz
997,Autumn,Walker
998,Naomi,Kelly


### Store

In [5]:
generateData = "genData.json"

ROOT = os.path.dirname(os.getcwd()) + '/resource/' + generateData

storeDict = {}

with open(ROOT, 'r') as file:
    data = json.load(file)
    data = data["Store"]

listAdd = []
for dict in data:
    listAdd.append(data[dict])

    
storeDf = pandas.DataFrame.from_records(listAdd)
storeDf

,Name,Address,City,Texas,Zip Code
0,South Austin Drive,102 South Austin Drive,Austin,Texas,72034
1,Western Drive,111 Western Drive,Houston,Texas,99345
2,North Lake,880 North Lake Drive,Dallas,Texas,14783


### Employee

In [6]:
generateData = "genData.json"

ROOT = os.path.dirname(os.getcwd()) + '/resource/' + generateData

with open(ROOT, 'r') as file:
    data = json.load(file)
    data = data["Employee"]

# get each stores employees
stores = []
for store in data:
    stores.append(data[store])


In [7]:
'''
{'one': {'EmployeeID': '1', 'ManagerID': 'Null', 'FirstName': 'Tom', 'LastName': 'Bates', 'StoreID': '1'}, 'two': {
'''

# Transpose json into wanted shape
df = pandas.DataFrame(stores[0]).T
# remove the unwanted index
stores.pop(0)

# merge wanted df into new df
for data in stores:
    dfNew = pandas.DataFrame(data).T
    df = pandas.concat([df,dfNew])

Employeedf = df
Employeedf.reset_index(drop=True, inplace=True)


### Product

In [8]:
generateData = "genData.json"

ROOT = os.path.dirname(os.getcwd()) + '/resource/' + generateData

with open(ROOT, 'r') as file:
    data = json.load(file)
    data = data["Product"]

# get each stores employees
product = []
for products in data:
    product.append(data[products])

productDF = pandas.DataFrame.from_records(product).T

productDF = productDF.rename(columns={0: 'Product', 1: 'Price'})

productDF['productID'] = productDF.reset_index().index + 1


### Full Table DF - Store

In [9]:
# Generate random integers between 0 and 10 for each row
random_counts = np.random.randint(0, 21, size=len(storeDf))
# random_counts

while storeDf.shape[0] < ROWCOUNT:
    
    # for each row number, count # of rows to add to df
    for row, count in enumerate(random_counts):
        
        # checker to prevent concatenating an empty list,
        if count > 0:
        
            # storeDf.concat([storeDf.loc[[i]] * count])
            row_to_duplicate = storeDf.loc[[row]]

            # Duplicate the row 2 times
            duplicated_row = pandas.concat([row_to_duplicate] * count, ignore_index=True)

            # Create a new DataFrame with the duplicated rows
            storeDf = pandas.concat([storeDf, duplicated_row], ignore_index=True)

# limit to 1000 rows
storeDf = storeDf.iloc[:ROWCOUNT]

### Full Table DF - Employee

In [10]:
# Generate random integers between 0 and 10 for each row
random_counts = np.random.randint(0, 21, size=len(Employeedf))
random_counts


array([16, 11,  3, 20,  8, 10, 18, 17,  7, 13,  4,  8,  8,  1])

In [11]:

while Employeedf.shape[0] < ROWCOUNT:
    
    # for each row number, count # of rows to add to df
    for row, count in enumerate(random_counts):
        # storeDf.concat([storeDf.loc[[i]] * count])
        
        # checker to prevent concatenating an empty list,
        if count > 0:
                
            row_to_duplicate = Employeedf.loc[[row]]

            # Duplicate the row 2 times
            duplicated_row = pandas.concat([row_to_duplicate] * count, ignore_index=True)

            # Create a new DataFrame with the duplicated rows
            Employeedf = pandas.concat([Employeedf, duplicated_row], ignore_index=True)

# limit to 1000 rows
Employeedf = Employeedf.iloc[:ROWCOUNT]

# Full table Df - Product

In [12]:
# Generate random integers between 0 and 10 for each row
random_counts = np.random.randint(0, 21, size=len(productDF))
random_counts


while productDF.shape[0] < ROWCOUNT:
    
    # for each row number, count # of rows to add to df
    for row, count in enumerate(random_counts):
        
        # checker to prevent concatenating an empty list,
        if count > 0:
                
            # storeDf.concat([storeDf.loc[[i]] * count])
            row_to_duplicate = productDF.loc[[row]]
                
            # Duplicate the row
            duplicated_row = pandas.concat([row_to_duplicate] * count, ignore_index=True)
            
            # Create a new DataFrame with the duplicated rows
            productDF = pandas.concat([productDF, duplicated_row], ignore_index=True)

# limit to 1000 rows
productDF = productDF.iloc[:ROWCOUNT]

## Concat ALl Tables into 1 table

In [13]:
fullTableDF = pandas.concat([customerDF,storeDf], axis=1)
fullTableDF = pandas.concat([fullTableDF,Employeedf], axis=1)
fullTableDF = pandas.concat([fullTableDF,productDF], axis=1)

In [26]:
# Set today's date
date = datetime.today().date()

# Define the time ranges with higher frequencies
morning_peak_start = 7.5  # 7:30am
morning_peak_end = 9
afternoon_peak_start = 15.5  # 3:30pm
afternoon_peak_end = 17

times = []

for _ in range(ROWCOUNT):
    # 40% chance of morning peak
    if np.random.rand() < 0.4:  
        hour = np.random.uniform(morning_peak_start, morning_peak_end)
    # 30% chance of afternoon peak
    elif np.random.rand() < 0.7:  
        hour = np.random.uniform(afternoon_peak_start, afternoon_peak_end)
    # 40% chance of random time between 6am and 9pm
    else:  
        hour = np.random.uniform(6, 21)

    minute = np.random.uniform(0, 60)
    time = datetime.combine(date, datetime.min.time()) + timedelta(hours=int(hour), minutes=int(minute))
    times.append(time)

# Create the pandas DataFrame
df = pd.DataFrame({'date': [date] * len(times), 'time': times})
dfDateTime = df.drop('date', axis=1)

In [27]:
fullTableDF = pandas.concat([fullTableDF,dfDateTime], axis=1)

In [28]:
fullTableDF

,First,Last,Name,Address,City,Texas,Zip Code,EmployeeID,ManagerID,FirstName,LastName,StoreID,Product,Price,productID,time
0,Luna,Parker,South Austin Drive,102 South Austin Drive,Austin,Texas,72034,1,Null,Tom,Bates,1,Hot Cocoa,4.5,1,2024-10-15 08:26:00
1,Gianna,Evans,Western Drive,111 Western Drive,Houston,Texas,99345,2,1,Alex,Dune,1,Espresso,3,2,2024-10-15 07:59:00
2,Sarah,Parker,North Lake,880 North Lake Drive,Dallas,Texas,14783,3,1,Frank,Watts,1,Ristretto,3.5,3,2024-10-15 15:36:00
3,Natalia,Thompson,South Austin Drive,102 South Austin Drive,Austin,Texas,72034,4,1,Daniel,Farmer,1,Americano,4,4,2024-10-15 15:29:00
4,Bella,Mitchell,South Austin Drive,102 South Austin Drive,Austin,Texas,72034,5,Null,Toby,Harts,2,Latte,5.5,5,2024-10-15 16:12:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Riley,Smith,South Austin Drive,102 South Austin Drive,Austin,Texas,72034,10,Null,Arthur,Alan,3,Cafe au Lait,4,19,2024-10-15 15:54:00
996,Chloe,Cruz,South Austin Drive,102 South Austin Drive,Austin,Texas,72034,10,Null,Arthur,Alan,3,Cafe au Lait,4,19,2024-10-15 11:11:00
997,Autumn,Walker,South Austin Drive,102 South Austin Drive,Austin,Texas,72034,10,Null,Arthur,Alan,3,Hot Cocoa,4.5,1,2024-10-15 08:22:00
998,Naomi,Kelly,South Austin Drive,102 South Austin Drive,Austin,Texas,72034,10,Null,Arthur,Alan,3,Hot Cocoa,4.5,1,2024-10-15 15:03:00


## GCP Push

In [29]:
fullTableDF.columns = ['CustomerFirstName', 'CustomerLastName', 'StoreName', 'StoreAddress','City','State','ZipCode','EmployeeId','ManagerId','EmployeeFirstName','EmployeeLastName','StoreId','ProductName','Price','ProductId','SaleDate']

In [30]:
fullTableDF

,CustomerFirstName,CustomerLastName,StoreName,StoreAddress,City,State,ZipCode,EmployeeId,ManagerId,EmployeeFirstName,EmployeeLastName,StoreId,ProductName,Price,ProductId,SaleDate
0,Luna,Parker,South Austin Drive,102 South Austin Drive,Austin,Texas,72034,1,Null,Tom,Bates,1,Hot Cocoa,4.5,1,2024-10-15 08:26:00
1,Gianna,Evans,Western Drive,111 Western Drive,Houston,Texas,99345,2,1,Alex,Dune,1,Espresso,3,2,2024-10-15 07:59:00
2,Sarah,Parker,North Lake,880 North Lake Drive,Dallas,Texas,14783,3,1,Frank,Watts,1,Ristretto,3.5,3,2024-10-15 15:36:00
3,Natalia,Thompson,South Austin Drive,102 South Austin Drive,Austin,Texas,72034,4,1,Daniel,Farmer,1,Americano,4,4,2024-10-15 15:29:00
4,Bella,Mitchell,South Austin Drive,102 South Austin Drive,Austin,Texas,72034,5,Null,Toby,Harts,2,Latte,5.5,5,2024-10-15 16:12:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Riley,Smith,South Austin Drive,102 South Austin Drive,Austin,Texas,72034,10,Null,Arthur,Alan,3,Cafe au Lait,4,19,2024-10-15 15:54:00
996,Chloe,Cruz,South Austin Drive,102 South Austin Drive,Austin,Texas,72034,10,Null,Arthur,Alan,3,Cafe au Lait,4,19,2024-10-15 11:11:00
997,Autumn,Walker,South Austin Drive,102 South Austin Drive,Austin,Texas,72034,10,Null,Arthur,Alan,3,Hot Cocoa,4.5,1,2024-10-15 08:22:00
998,Naomi,Kelly,South Austin Drive,102 South Austin Drive,Austin,Texas,72034,10,Null,Arthur,Alan,3,Hot Cocoa,4.5,1,2024-10-15 15:03:00
